# Data preprocessing

In [ ]:
# Import libraries
import pandas as pd
import os
import csv
import billboard
import json
import re
import lyricsgenius
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import requests
pd.options.mode.chained_assignment = None

### Genius and Spotify APIs' settings

In [ ]:
# Genius token
genius_token = os.environ.get('genius_token')

# Spotify authorization
spotify_client_id = os.environ.get('spotify_client_id')
spotify_client_secret = os.environ.get('spotify_client_secret')
client_credentials_manager = SpotifyClientCredentials(client_id=spotify_client_id, client_secret=spotify_client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

### CSV data

In [ ]:
# Loading data and adding year columns and wrapping up billboard songs (1950-2015)
wd = os.getcwd()
data_path = os.chdir(wd + '\data')
all_csv_files = os.listdir(data_path)
songs_50_15 = []

for filename in all_csv_files:
    songs = pd.read_csv(filename, header = 0, sep = ';')
    year = os.path.basename(filename)[:-4]
    songs['Year'] = year
    songs_50_15.append(songs)

### billboard library data

In [ ]:
# Downloading data from years 2016-2020 from billboard library
playlist = 'hot-100-songs'
dates = [2016, 2017, 2018, 2019, 2020]
songs_billboard = []
for y in dates:
    songs = billboard.ChartData(playlist, date = None, year = y, fetch = True, timeout = 25)
    songs_billboard.append(songs)

In [ ]:
# Preprocessing data from years 2016-2020
song_titles, artists = [[] for i in range(2)]

for year in songs_billboard:
    for playlist in year:
        song = str(playlist).split(' by ')
        song_titles.append(song[0].strip("\'"))
        artists.append(song[1])
        
# Adding missed song on 87th position in 2016 chart
song_titles.insert(86, 'All the Way Up')
artists.insert(86, 'Fat Joe, Remy Ma and Jay-Z featuring French Montana and Infared')

### Concatenating CSV and billboard library data

In [ ]:
# pd.set_option("display.max_rows", None, "display.max_columns", None)
# Splitting list into same-length elements lists
def list_split(l, n):
    n = max(1, n)
    return (l[i:i+n] for i in range(0, len(l), n))

split_artists = list(list_split(artists, 100))
split_song_titles = list(list_split(song_titles, 100))

# Creating list of dictionaries for each year to convert them into pandas DataFrame
def dicts_of_dates(lst_of_dates):
    songs = []
    for i in range(len(lst_of_dates)):
        ranger = list(range(1, 101))
        year = lst_of_dates[i]
        dictionary = {'Position': ranger, 'Artist': split_artists[i], 'Song Title': split_song_titles[i], 'Year': [dates[i] for year in range(1, 101)]}
        as_pandas_df = pd.DataFrame(dictionary)
        songs.append(as_pandas_df)
    return songs

# Concatenate all songs from years 1950 - 2020
songs_16_20 = dicts_of_dates(dates)
all_songs = songs_50_15 + songs_16_20
all_time_billboard_wrap_up = pd.concat(all_songs)
all_time_billboard_wrap_up.reset_index(drop = True, inplace = True) # reset indexing
display(all_time_billboard_wrap_up)

In [ ]:
# all_time_billboard_wrap_up dataframe nans checking
nans_table = [all_time_billboard_wrap_up['Position'].isna().any(), all_time_billboard_wrap_up['Artist'].isna().any(), all_time_billboard_wrap_up['Song Title'].isna().any(), all_time_billboard_wrap_up['Year'].isna().any()]
nans_proj = pd.DataFrame({'Column': ['Position', 'Artist', 'Song Title', 'Year'], 'Is nan?': nans_table}).set_index('Column')
display(nans_proj)

In [ ]:
# pd.set_option("display.max_rows", None, "display.max_columns", None)
# Spliting authors by 'and' and 'Featuring' to make api searching easier and more efficient
artists = list(all_time_billboard_wrap_up.loc[:,'Artist'])
split_artists = []
for artist in artists:
    if any(re.findall(r'and|featuring|feat.|Featuring|Feat.&', str(artist))):
        result = re.split(r'and|featuring|feat.|Featuring|Feat.&', str(artist))[0].strip()
        split_artists.append(result)
    else:
        split_artists.append(artist)

all_time_billboard_wrap_up['Split Names'] = split_artists
display(all_time_billboard_wrap_up)

### Genius data into json files

In [ ]:
# lyricsgenius settings
genius = lyricsgenius.Genius(genius_token, timeout = 30, retries = 3)
songs = list(all_time_billboard_wrap_up.loc[:, 'Song Title'])
artists = list(all_time_billboard_wrap_up.loc[:,'Split Names'])
jsons_path = wd + '\\jsons\\'

# Saving lyrics to jsons function
def return_lyrics_json(song, artist, json_name):
    song_lyrics = genius.search_song(song, artist)
    with open(jsons_path + json_name, 'w', encoding = 'UTF-8') as f:
        json.dump({'lyrics': song_lyrics.lyrics}, f, ensure_ascii = False, indent = 4)

# No lyrics function (only instrumental songs)
def no_lyrics(json_name):
    with open(jsons_path + json_name, 'w', encoding = 'UTF-8') as f:
        json.dump({'lyrics': ""}, f, ensure_ascii = False, indent = 4)  

In [ ]:
# Applying return_lyrics_json and no_lyrics functions on lyrics:
def json_data():
    error_list = []
    for i in range(len(songs)):
        try:
            return_lyrics_json(songs[i], artists[i], str(i)+'.json')
        except AttributeError:
            no_lyrics(str(i)+'.json')
            error_list.append(i)
# json_data()

In [ ]:
# pd.set_option("display.max_rows", None, "display.max_columns", None)
# err based on error_list
# Fixed 6140 Beyonce 7/11 (date format in csv)
err = [12, 23, 25, 53, 60, 74, 84, 100, 101, 103, 111, 116, 141, 160, 173, 190, 196, 210, 233, 239, 266, 277, 334, 365, 380, 385, 386, 401, 428, 439, 440, 448, 501, 515, 519, 537, 554, 565, 604, 616, 619, 633, 690, 694, 696, 708, 723, 771, 774, 784, 800, 816, 832, 838, 853, 862, 866, 871, 906, 920, 937, 969, 991, 1039, 1111, 1126, 1150, 1180, 1211, 1245, 1277, 1381, 1387, 1397, 1400, 1422, 1454, 1537, 1542, 1569, 1598, 1613, 1615, 1668, 1801, 1807, 1822, 1850, 1873, 1895, 1940, 1965, 1982, 2027, 2095, 2144, 2167, 2177, 2189, 2208, 2269, 2350, 2370, 2400, 2528, 2559, 2603, 2633, 2690, 2738, 2791, 2835, 2843, 3140, 3334, 3460, 3742, 3908, 3952, 4121, 4169, 4171, 4180, 4208, 4244, 4245, 4280, 4351, 4361, 4387, 4518, 4579, 4760, 4777, 4823, 5134, 5235]

### Deleting empty or invalid json files

In [ ]:
# Deleting empty json files
# Adding string '.json' to err
json_delete = [str(j) + '.json' for j in err]
# Deteling json files that are in json_delete list and bigger than 5KB
all_json_files = os.listdir(jsons_path)
for x in all_json_files:
    filepath = jsons_path + str(x)
    json_size = os.path.getsize(filepath)
    if (json_size > 5 * 1024) or (x in err): #error_list
        os.remove(filepath)
# Selecting all_billboard_wrap_up data corresponding with actual list of jsons
ordered_jsons = sorted([int(x[:-5]) for x in os.listdir(jsons_path)])
all_time_billboard_wrap_up_cleaned = all_time_billboard_wrap_up.iloc[ordered_jsons]

### Spotify IDs

In [ ]:
# pd.set_option("display.max_rows", None, "display.max_columns", None)
# Function that enables finding Spotify ID from spotipy
def find_spotify_id(artist, track):
    try:
        track_id = spotipy.search(q='artist:' + artist + ' track:' + track, type='track')
        return track_id['tracks']['items'][0]['id']
    except IndexError:
        return ''    
all_time_billboard_wrap_up_cleaned['Spotify ID'] = [find_spotify_id(all_time_billboard_wrap_up_cleaned['Split Names'].iloc[i], all_time_billboard_wrap_up_cleaned['Song Title'].iloc[i]) for i in range(len(all_time_billboard_wrap_up_cleaned))]
display(all_time_billboard_wrap_up_cleaned)

In [ ]:
# pd.set_option("display.max_rows", None, "display.max_columns", None)
sum(all_time_billboard_wrap_up_cleaned['Spotify ID'] == '')

In [ ]:
# Check artist and song name for n Spotify IDs
spotify_ids = all_time_billboard_wrap_up_cleaned['Spotify ID'].tolist()
spotify_artists = []
spotify_songs = []
for i in range(0, len(spotify_ids)):
    if spotify_ids[i] != '':
        track = spotipy.track(spotify_ids[i])
        song_name = track['name']
        spotify_songs.append(song_name)
        artist = track['album']['artists'][0].get('name')
        spotify_artists.append(artist)
    else:
        song_name, artist = '', ''
        spotify_songs.append(song_name)
        spotify_artists.append(artist)

In [ ]:
all_time_billboard_wrap_up_cleaned['Spotify Artist'] = spotify_artists
all_time_billboard_wrap_up_cleaned['Spotify Song'] = spotify_songs

In [ ]:
all_time_billboard_wrap_up_cleaned_sp_data = all_time_billboard_wrap_up_cleaned[all_time_billboard_wrap_up_cleaned['Spotify Artist'].isin(all_time_billboard_wrap_up_cleaned['Artist'])]
display(all_time_billboard_wrap_up_cleaned_sp_data)

### Spotify audio features

In [ ]:
for i in all_time_billboard_wrap_up_cleaned_sp_data['Spotify ID']:
    df = pd.DataFrame(sp.audio_features(i))
    all_time_billboard_wrap_up_cleaned_sp_data = all_time_billboard_wrap_up_cleaned_sp_data.merge(df, how = 'left', left_on = 'Spotify ID', right_on = 'id')
display(all_time_billboard_wrap_up_cleaned_sp_data)